## MASSIVE ALGORITHMS

### 1. Data Import

In [1]:
#import os
#import zipfile

In [2]:
#os.environ['KAGGLE_USERNAME'] = "melissarizzi"
#os.environ['KAGGLE_KEY'] = "3ed913e7329a3117a254e67179c0f8bb"

In [3]:
#!pip install kaggle

In [4]:
#!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

In [5]:
#with zipfile.ZipFile("amazon-books-reviews.zip", 'r') as zip_ref:
#    zip_ref.extractall("amazon_books_data")

### 2. Data PreProcessing

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("MapReduce").getOrCreate()

In [3]:
data = spark.read.csv("amazon_books_data/Books_rating.csv", header=True, inferSchema=True)

In [4]:
data.show(5)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

In [5]:
df = data.select("Id", 'Title', "User_id", "review/score",'review/text').withColumnRenamed("review/score", "score")

In [6]:
df.show()

+----------+--------------------+--------------+-----+--------------------+
|        Id|               Title|       User_id|score|         review/text|
+----------+--------------------+--------------+-----+--------------------+
|1882931173|Its Only Art If I...| AVCGYZL8FQQTD|  4.0|This is only for ...|
|0826414346|Dr. Seuss: Americ...|A30TK6U7DNS82R|  5.0|I don't care much...|
|0826414346|Dr. Seuss: Americ...|A3UH4UZ4RSVO82|  5.0|"If people become...|
|0826414346|Dr. Seuss: Americ...|A2MVUWT453QH61|  4.0|Theodore Seuss Ge...|
|0826414346|Dr. Seuss: Americ...|A22X4XUPKF66MR|  4.0|"Philip Nel - Dr....|
|0826414346|Dr. Seuss: Americ...|A2F6NONFUDB6UK|  4.0|"""Dr. Seuss: Ame...|
|0826414346|Dr. Seuss: Americ...|A14OJS0VWMOSWO|  5.0|Theodor Seuss Gie...|
|0826414346|Dr. Seuss: Americ...|A2RSSXTDZDUSH4|  5.0|"When I recieved ...|
|0826414346|Dr. Seuss: Americ...|A25MD5I2GUIW6W|  5.0|"Trams (or any pu...|
|0826414346|Dr. Seuss: Americ...|A3VA4XFS5WNJO3|  4.0|As far as I am aw...|
|0829814000|

In [7]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- score: string (nullable = true)
 |-- review/text: string (nullable = true)



In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

df = df.withColumn("score", col("score").cast(DoubleType()))
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- score: double (nullable = true)
 |-- review/text: string (nullable = true)



In [10]:
from pyspark.sql.functions import min, max

df.select(min(col("score")).alias("min_score"), max(col("score")).alias("max_score")).show()

+---------+----------+
|min_score| max_score|
+---------+----------+
|      1.0|1.295568E9|
+---------+----------+



In [11]:
df.select("score").distinct().show(100)

+-----------+
|      score|
+-----------+
|        1.0|
|        4.0|
|      19.95|
|       NULL|
|        3.0|
|        2.0|
|        5.0|
|      327.0|
| 1.295568E9|
|1.2089952E9|
|  1.21176E9|
+-----------+



In [12]:
df = df.filter((col("score") >= 1) & (col("score") <= 5))
df.select(min("score").alias("min_score"), max("score").alias("max_score")).show()

+---------+---------+
|min_score|max_score|
+---------+---------+
|      1.0|      5.0|
+---------+---------+



#### 2.1 Missing data

In [32]:
df_clean = df.dropna()

# Mostra il risultato
df_clean.show()


+----------+--------------------+--------------+-----+--------------------+
|        Id|               Title|       User_id|score|         review/text|
+----------+--------------------+--------------+-----+--------------------+
|1882931173|Its Only Art If I...| AVCGYZL8FQQTD|  4.0|This is only for ...|
|0826414346|Dr. Seuss: Americ...|A30TK6U7DNS82R|  5.0|I don't care much...|
|0826414346|Dr. Seuss: Americ...|A3UH4UZ4RSVO82|  5.0|"If people become...|
|0826414346|Dr. Seuss: Americ...|A2MVUWT453QH61|  4.0|Theodore Seuss Ge...|
|0826414346|Dr. Seuss: Americ...|A22X4XUPKF66MR|  4.0|"Philip Nel - Dr....|
|0826414346|Dr. Seuss: Americ...|A2F6NONFUDB6UK|  4.0|"""Dr. Seuss: Ame...|
|0826414346|Dr. Seuss: Americ...|A14OJS0VWMOSWO|  5.0|Theodor Seuss Gie...|
|0826414346|Dr. Seuss: Americ...|A2RSSXTDZDUSH4|  5.0|"When I recieved ...|
|0826414346|Dr. Seuss: Americ...|A25MD5I2GUIW6W|  5.0|"Trams (or any pu...|
|0826414346|Dr. Seuss: Americ...|A3VA4XFS5WNJO3|  4.0|As far as I am aw...|
|0829814000|

What stands out right away, especially for the purpose of our analysis, is that there are many missing values in the User_id variable. One possible reason for this could be that users who leave reviews but are not registered don’t have a user ID. Our goal is to identify baskets of items purchased by the same users, but without the user ID, this analysis cannot be conducted. We explored the possibility of using profile names instead, by assigning a dummy ID to users with the same name. However, we were aware that this might not provide accurate results due to potential name duplication. Moreover, there were more missing profile names than missing user IDs, which made this solution unfeasible. After considering our options, we ultimately decided to **drop the missing values**, as we couldn’t identify a suitable method to replace them.

In [33]:
# Ottieni il numero di righe
num_righe1 = df.count()
num_righe = df_clean.count()
print(f"Numero di righe: {num_righe1}")
print(f"Numero di righe post cleaning: {num_righe}")

# Ottieni il numero di colonne
num_colonne = len(df_clean.columns)
num_colonne1 = len(df.columns)
# Mostra le dimensioni del dataset
print(f"Numero di colonne: {num_colonne1}")
print(f"Numero di colonne post cleaning : {num_colonne}")


Numero di righe: 2981912
Numero di righe post cleaning: 2420235
Numero di colonne: 5
Numero di colonne post cleaning : 5


#### 2.2 Data Duplicates

In [34]:
# Rimuovi righe duplicate su tutte le colonne
df_clean = df_clean.dropDuplicates()

num_righe2 = df_clean.count()
print(f"Numero di righe post no duplicati: {num_righe2}")

Numero di righe post no duplicati: 2398220


Up until now, we’ve performed a general cleaning of the dataset. From here on, we’ll focus exclusively on the three columns that are relevant to our analysis (Id, User_id, and score), forming a new dataset: df_short.

In [35]:
# Remove useless columns
df_short = df_clean.select("Id", "User_id","score")

In [36]:
df_short.show(5)

+----------+--------------+-----+
|        Id|       User_id|score|
+----------+--------------+-----+
|0809080699|A29LG535LJRITI|  5.0|
|0671551345|A3CLKX8W3F1L1D|  3.0|
|0671551345| A4FX5YCJA630V|  3.0|
|B000MCKQRS| A1237ROTM7659|  4.0|
|B000890HE2|A2JYUAIAUYXCQN|  5.0|
+----------+--------------+-----+
only showing top 5 rows



In [37]:
# Check duplicated rows for the three considered variables
df_short= df_short.dropDuplicates()

In [38]:
from pyspark.sql import functions as F

# Trova i duplicati basati su Id e User_id
duplicati = df_short.groupBy("Id", "User_id").count().filter("count > 1")

# Mostra i duplicati
duplicati.show()


+----------+--------------+-----+
|        Id|       User_id|count|
+----------+--------------+-----+
|0451518713|A2TZZQUHX0PVN4|    2|
|B000K0DB8I|A2OJH3S0SUNVGE|    2|
|B000Q56SO6|A23LDF7TIIKTCY|    2|
|B000GSKM2M|A1RJD10TTI568L|    2|
|B0006W43TQ| A81F0YW06W5VQ|    2|
|B000PEC9LO| AESOLJBO0EK6M|    2|
|185723569X|A2E87TFJLKP87Y|    2|
|B000NGSJ36| ATFXX7NGPHA7N|    2|
|0451521196|A3PWVRILWO2SCH|    2|
|0613175719|A1FDV3WPOHREY9|    2|
|B000JGUH3A|A1DVZR8EYXFK8K|    2|
|B000ILIJE0| ACFOTAO0A7E3Y|    2|
|B000JPCAFO| AMG5L1IFX5BNB|    2|
|1558321527| A2C27IQUH9N1Z|    2|
|B000N6R3FG|A2QAOP52OEB5EV|    2|
|1594862265|A33FY1547CZHHF|    2|
|0791051323| A5O5SDU4HCYPX|    2|
|0774033096|A1D2C0WDCSHUWZ|    2|
|B000MTRTTE|A2FTHCGH06O4Y5|    2|
|1581127197|A3I8IYYQEC39T9|    2|
+----------+--------------+-----+
only showing top 20 rows



In [39]:
from pyspark.sql import functions as F

# Calcola la media del punteggio per ogni (Id, User_id)
score_mean = df_short.groupBy('Id', 'User_id').agg(F.mean('score').alias('mean_score'))

# Unisci la media calcolata al dataframe originale
df_final = df_short.join(score_mean, on=['Id', 'User_id'], how='left')

# Mostra i primi 5 record
df_final.show(5)


+----------+--------------+-----+----------+
|        Id|       User_id|score|mean_score|
+----------+--------------+-----+----------+
|1597400602|A3DKP67DK28RUB|  5.0|       5.0|
|B0007H4QBK|A3MVU8X8EC9VRT|  5.0|       5.0|
|B0007H4QBK| AFP82QXWXAG2V|  5.0|       5.0|
|B000MCKQRS|A1TQL7XMTVF4JG|  3.0|       3.0|
|B000O3QCH8|A1T97PDD7JYCUA|  1.0|       1.0|
+----------+--------------+-----+----------+
only showing top 5 rows



In [40]:
df_final = df_final.select('Id','User_id', 'mean_score')

In [41]:
df_final = df_final.dropDuplicates()

In [42]:
df_final.show(5)

+----------+--------------+----------+
|        Id|       User_id|mean_score|
+----------+--------------+----------+
|0001047604|A1ZQ1LUQ9R6JHZ|       5.0|
|0001047655|A12N9YU5K516JF|       4.0|
|0001047655|A1EB4FLIXNX0LK|       2.0|
|0001047655|A1NS4974T51EU1|       5.0|
|0001047655|A2C8IVS3AEH96R|       1.0|
+----------+--------------+----------+
only showing top 5 rows



#### 2.3 Rating means

In [43]:
# Check integrity of data
from pyspark.sql.functions import min, max

df.select(min("score").alias("min_score"), max("score").alias("max_score")).show()

+---------+---------+
|min_score|max_score|
+---------+---------+
|      1.0|      5.0|
+---------+---------+



We want to calculate the average score for each book (Id) to see if consistency is maintained after creating the subsample.

- Overall mean score:

In [44]:
# Converti la colonna 'score' in tipo numerico (Double)
df_final = df_final.withColumn("mean_score", F.col("mean_score").cast("double"))

# Calcola la media della colonna 'score'
media_score = df_final.agg(F.avg("mean_score")).collect()[0][0]

# Stampa il risultato
print(f"La media della colonna 'score' è: {media_score}")

La media della colonna 'score' è: 4.22386130919595


- Mean score for each item:

In [46]:
# Raggruppa per 'Id' e calcola la media della colonna 'score'
score_per_id = df_final.groupBy("Id").agg(F.avg("mean_score").alias("avg_score"))

# Mostra il risultato
score_per_id.show()

+----------+------------------+
|        Id|         avg_score|
+----------+------------------+
|0027861317|             4.625|
|0028622480|               4.0|
|0029267358|               4.0|
|0060929081|              4.24|
|0071409807|              3.75|
|0132414074|               4.0|
|0140187855|               4.5|
|0140440488|4.2631578947368425|
|0192801996|            4.3125|
|0192892355|3.4285714285714284|
|0195047672| 4.714285714285714|
|0195142853|             4.875|
|0198661886|               4.5|
|0201180758| 4.615384615384615|
|0201433206|               4.5|
|0201700344|2.5714285714285716|
|0271019441|               4.5|
|0281054657|               4.4|
|0300083238|            3.8125|
|0304352616|               5.0|
+----------+------------------+
only showing top 20 rows



In [47]:
# Filtra gli Id con una media maggiore di 5
score_per_id_above_5 = score_per_id.filter(F.col("avg_score") > 5)

# Mostra il risultato
score_per_id_above_5.show()


+---+---------+
| Id|avg_score|
+---+---------+
+---+---------+

